In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
from matplotlib.pyplot import imread
import codecs
from IPython.display import HTML
import re

movies = pd.read_csv('dataset.csv')

movies = movies.drop(['desc_snippet','genre','publisher','game_description','languages','recent_reviews','types','achievements','minimum_requirements','recommended_requirements','original_price','discount_price'], axis=1)

movies['mature_content'] = movies['mature_content'].fillna(0)
for i in range(movies.mature_content.size):
    if(movies.mature_content[i]!=0):
        movies.mature_content[i] = 1
    else:
        movies.mature_content[i] = 0


movies.dropna(inplace=True)
movies.reset_index(drop=True, inplace=True)

movies['popular_tags'] = (movies['game_details']+movies['popular_tags'])

my_list=[]
for i in range(movies['popular_tags'].size):
    movies['popular_tags'][i] = (movies['popular_tags'][i].split(",")) 

for i in range(movies['release_date'].size):
        movies['release_date'][i] = movies['release_date'][i][-4:]


for i in range(movies['all_reviews'].size):
    if(type(movies['all_reviews'][i]) == int):
        pass
    elif(movies['all_reviews'][i][0] >= '0' and movies['all_reviews'][i][0] <='9'):
        movies['all_reviews'][i]=0
    elif(movies['all_reviews'][i][0] == 'N' or movies['all_reviews'][i][5] == 'N'):
        movies['all_reviews'][i]=0
    else:
        movies['all_reviews'][i] = int(re.findall('\d*%', movies['all_reviews'][i])[0][0:2])
movies = movies[movies['all_reviews']!=0]

movies.reset_index(drop=True, inplace=True)

for i,j in zip(movies['popular_tags'],movies.index):
    list2=[]
    list2=i
    list2.sort()
    movies.loc[j,'popular_tags']=str(list2)
movies['popular_tags'] = movies['popular_tags'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['popular_tags'] = movies['popular_tags'].str.split(',')

popularTagList = []
for index, row in movies.iterrows():
    popular_tags = row["popular_tags"]
    
    for popular_tag in popular_tags:
        if popular_tag not in popularTagList:
            popularTagList.append(popular_tag)

def binary(genre_list):
    binaryList = []
    
    for genre in popularTagList:
        if genre in genre_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

movies['popular_tags_bin'] = movies['popular_tags'].apply(lambda x: binary(x))
movies['popular_tags_bin']

ml=[]
for i in movies['popular_tags_bin']:
    ml.append(np.array(i))
ml = np.array(ml)

df2 = movies[['url','name','all_reviews','release_date','developer','popular_tags','mature_content']]

def numeric_till_String(test_str):
    temp = 0
    while (test_str[temp] != '/'): 
        if(temp == len(test_str) -1 ):
            break
        temp+=1
    
    if(test_str[0 : temp] == ''):
        return -1
           
    return int(test_str[0 : temp])


df2_appid = []

for i in range(movies['url'].size):
    df2_appid.append(numeric_till_String(movies['url'][i][35:]))

df2['appid'] = df2_appid

print(df2.head())




<>:49: SyntaxWarning: invalid escape sequence '\d'
<>:49: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14412\3894378307.py:49: SyntaxWarning: invalid escape sequence '\d'
  movies['all_reviews'][i] = int(re.findall('\d*%', movies['all_reviews'][i])[0][0:2])


                                                 url  \
0    https://store.steampowered.com/app/379720/DOOM/   
1  https://store.steampowered.com/app/578080/PLAY...   
2  https://store.steampowered.com/app/637090/BATT...   
3    https://store.steampowered.com/app/221100/DayZ/   
4  https://store.steampowered.com/app/8500/EVE_On...   

                            name all_reviews release_date  \
0                           DOOM          92         2016   
1  PLAYERUNKNOWN'S BATTLEGROUNDS          49         2017   
2                     BATTLETECH          71         2018   
3                           DayZ          61         2018   
4                     EVE Online          74         2003   

             developer                                       popular_tags  \
0          id Software  [Action, Atmospheric, Blood, Classic, Co-op, C...   
1     PUBG Corporation  [Action, BattleRoyale, Co-op, Competitive, Dif...   
2  Harebrained Schemes  [Action, Adventure, CharacterCustomizatio

In [ ]:
import pickle

pickle.dump(df2.to_dict(),open('meta_games_dict.pkl','wb'))

pickle.dump(ml,open('confusion_matrix_con.pkl','wb'))

json_object = json.dumps(df2.to_dict(), indent = 4)
  
# Writing to sample.json
with open("dataset.json", "w") as outfile:
    outfile.write(json_object)

with open("./../react-server/src/dataset.json", "w") as outfile:
    outfile.write(json_object)



In [1]:
import pandas as pd
import numpy as np
import pickle

# Load the dataset
with open('meta_games_dict.pkl', 'rb') as f:
    movies = pickle.load(f)

# Load the binary tag array (ml)
with open('confusion_matrix_con.pkl', 'rb') as f:
    ml = pickle.load(f)


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(ml, ml)

def get_recommendations(game_name, cosine_sim=cosine_sim, df=movies):
    # Find the index of the game
    idx = df[df['name'].str.lower() == game_name.lower()].index[0]
    
    # Get the pairwise similarity scores of all games with that game
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the games based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar games
    sim_scores = sim_scores[1:11]
    
    # Get the game indices
    game_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar games
    return df['name'].iloc[game_indices]


In [2]:
movies = pd.DataFrame(movies)

In [3]:
# change developer name of Counter-Strike: Global Offensive
movies.loc[movies['name'] == 'Counter-Strike: Global Offensive', 'developer'] = 'Valve'

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
# Function to get recommendations based on tags
def get_recommendations_by_tags(game_name, cosine_sim=cosine_sim, df=movies):
    idx = df[df['name'].str.lower() == game_name.lower()].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    game_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[game_indices]

# Function to get games from the same developer
def get_recommendations_by_developer(game_name, df=movies):
    developer = df[df['name'].str.lower() == game_name.lower()]['developer'].values[0]
    return df[df['developer'] == developer]['name']

# Function to get recent games
def get_recent_games(df=movies, top_n=10):
    recent_games = df.sort_values(by='release_date', ascending=False)
    return recent_games['name'].head(top_n)

# Function to get top-reviewed games
def get_top_reviewed_games(df=movies, top_n=10):
    top_reviewed = df.sort_values(by='all_reviews', ascending=False)
    return top_reviewed['name'].head(top_n)

# Example usage
game = "The Witcher® 3: Wild Hunt"
print("1",get_recommendations_by_tags(game))
print("\n2",get_recommendations_by_developer(game))
print("\n3",get_recent_games())
print("\n4",get_top_reviewed_games())


1 5198             The Witcher 3: Wild Hunt - Blood and Wine
47            The Witcher: Enhanced Edition Director's Cut
137      The Witcher 2: Assassins of Kings Enhanced Edi...
48                           Middle-earth™: Shadow of War™
5129            The Witcher 3: Wild Hunt - Hearts of Stone
15401                                 Lords Of The Fallen™
89                                       Fable Anniversary
1227                                       Game of Thrones
138      The Elder Scrolls IV: Oblivion® Game of the Ye...
459                       Middle-earth™: Shadow of Mordor™
Name: name, dtype: object

2 47           The Witcher: Enhanced Edition Director's Cut
137     The Witcher 2: Assassins of Kings Enhanced Edi...
271                      Thronebreaker: The Witcher Tales
351                             The Witcher® 3: Wild Hunt
5129           The Witcher 3: Wild Hunt - Hearts of Stone
5198            The Witcher 3: Wild Hunt - Blood and Wine
5233            The Witcher 3: 